Como treinar YOLOv3 usando Darknet no notebook Colab e otimizar os tempos de carregamento do tempo de execução da VM
Bem-vindo!
Este notebook Colab mostrará como:

Treine um modelo Yolo v3 usando Darknet usando a GPU Colab 12GB-RAM .
Transforme os notebooks Colab em uma ferramenta eficaz para trabalhar em projetos reais. Lidando com a desvantagem de um tempo de execução que explodirá a cada 12 horas no espaço!
Trabalhando diretamente dos arquivos em seu computador.
Configure seu notebook para instalar tudo o que você precisa e comece a treinar em cerca de um minuto (testado usando um conjunto de dados de 550 MB).
Receba seus pesos treinados diretamente em seu computador durante o treinamento. Enquanto o notebook está treinando, você pode verificar como está indo usando seus pesos treinados em seu computador.
Este notebook faz parte da postagem Como treinar o YOLOv3 usando a estrutura Darknet e otimizar os tempos de carregamento do tempo de execução da VM que eu encorajo você a visitar! Você encontrará uma explicação mais profunda sobre o Google Colab, as vantagens e as limitações dessa ótima ferramenta.
Estas são as etapas que seguiremos:
Configure o Google Drive e mapeie como unidade de rede .
Alguns utilitários para ajudar a fazer algumas tarefas.
Configure os pré-requisitos no tempo de execução.
Verifique a instalação do CUDA na VM de tempo de execução.
Instale o cuDNN.
Clone e compile Darknet. Usaremos um repositório baseado no repositório Darknet de AlexeyAB . Apliquei algumas alterações para possibilitar o carregamento de arquivos /My Drive/e reduzi o número de logs no console para acelerar o notebook.
Vamos verificar se tudo funciona muito bem.
Explicado como gerenciar seus arquivos YOLO em seu computador e será usado de forma transparente a partir deste notebook.
NOTA: Células com uma anotação # Not Necessary cell. Pode ser removido sem causar impacto. São apenas conteúdos explicativos.

Sem mais delongas, vamos começar!
ETAPA 0. Configure o tempo de execução para trabalhar com GPU
Queremos usar a aceleração de hardware da GPU de 12 GB de RAM !

Vá para > Menu > Tempo de execução > Configurar tipo de tempo de execução e selecione GPU no menu suspenso do acelerador de hardware

ETAPA 1. Conecte seus arquivos ao Google Drive
Para ter seus arquivos em seu computador local, você precisa instalar e configurar o Google Backup and Sync para manter uma pasta de seu Drive sincronizada com uma pasta em seu computador.

unidade de esquema.jpg

Crie uma pasta no seu Google Drive chamada darknet
Configure o Google Backup and Sync da seguinte maneira. Se você não fala espanhol, talvez aprenda algumas palavras! ;) Sync Drive.jpg
Após esta etapa, você terá uma pasta chamada darknet em seu computador local. Esta pasta é onde você trabalhará com arquivos em seu computador

DICA - Precisamos ter um bom desempenho baixando os dados do Drive para o Colab. Ter muitos arquivos na pasta raiz do Drive pode tornar as coisas um pouco mais lentas. É uma boa prática trabalhar com o Colab para mover todos os arquivos da pasta raiz para uma pasta

ETAPA 2. Conecte o notebook Colab ao Google Drive
Agora vamos mapear sua pasta do Google Drive. Esta primeira etapa é a única que exigirá sua interação manual toda vez que você executar seu notebook.

Execute a célula a seguir (clique no botão Play ou pressione CTRL + ENTER) e clique no link para autorizar seu notebook a acessar seu Google Drive.
Cole o código que o Google fornecerá a você e pressioneenter
[ ]
# This cell imports the drive library and mounts your Google Drive as a VM local drive. You can access to your Drive files 
# using this path "/content/gdrive/My Drive/"

from google.colab import drive
drive.mount('/content/gdrive')
Parabéns! Agora você pode acessar a pasta local do seu computador diretamente daqui!

Confira!

[ ]
# Not Necessary cell
# List the content of your local computer folder 
!ls -la "/content/gdrive/My Drive/darknet"
[ ]
!sudo apt-get install tree
[ ]
!tree /content/gdrive/My\ Drive/darknet/
NOTA DE ATUALIZAÇÃO
Se você deseja simplificar seus caminhos, pode usar um link simbólico:

!ln -s "/content/gdrive/My Drive/darknet/" /mydrive

Então você poderá acessar seus arquivos do Google Drive apenas usando /mydriveo caminho

CUIDADO: Todos os caminhos deste caderno estão sem link simbólico. Você terá que se lembrar de mudar o caminho em todos os lugares

Obrigado a Dennis Kashkin por esta recomendação!

[ ]
# Uncomment if you want to use Symbolic link
#!ln -s /content/gdrive/My\ Drive/darknet/ /mydrive
#!ls /mydrive
ETAPA 2. Verifique a versão de lançamento do CUDA
A Nvidia CUDA vem pré-instalada nos notebooks Colab. Agora vamos verificar a versão instalada.

ATENÇÃO: Talvez em algum momento desde o momento em que estou escrevendo estas linhas (9 de abril de 2019) a versão CUDA seja atualizada no Colab e você deva baixar outra versão do cuDNN na próxima etapa. Agora é a versão 10.0 e estamos usando cuDNN (cudnn-10.0-linux-x64-v7.5.0.56.tgz) de acordo

[ ]
# This cell can be commented once you checked the current CUDA version
# CUDA: Let's check that Nvidia CUDA is already pre-installed and which version is it. In some time from now maybe you 
!/usr/local/cuda/bin/nvcc --version

ETAPA 3. Instale o cuDNN de acordo com a versão atual do CUDA
Agora, você precisa baixar o cuDNN do site da Nvidia. Você precisará se inscrever no site.

Baixe o cuDNN do site da Nvidia

No momento, como temos o CUDA 10.0 pré-instalado no tempo de execução do Colab, você precisa baixar o cuDNN v7.5.0.56 para CUDA v10.0 - o arquivo é cudnn-10.0-linux-x64-v7.5.0.56.tgz
Em seu computador local, crie uma pasta chamada cuDNN em sua pasta local darknet . Copie o arquivo tgz para lá

[ ]
# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xzvf gdrive/My\ Drive/darknet/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2
ETAPA 4. Instalando o Darknet
Ótimo!! Temos todo o necessário para começar a trabalhar com a Darknet.

Este notebook funciona com uma versão ligeiramente modificada do darknet, baseada no repositório AlexeyAB Darknet. As alterações aplicadas ao repositório original são:

Permitindo usar espaços nos arquivos de configuração darknet obj.data . Necessário para trabalhar diretamente com o Google Drive.
Removendo alguns logs em cada época. O repositório original grava mais logs do que o notebook Colab pode sincronizar. Isso cria uma longa fila durante o treinamento. Esta versão mostra apenas os resultados após cada iteração.
Você pode dar uma olhada no código no repositório github

TRUQUE : Como queremos executar o notebook rapidamente toda vez, compilaremos o darknet apenas na primeira vez que executarmos este notebook. Em seguida, salvaremos a versão compilada em sua unidade. Para as execuções futuras, copiaremos o compilado em vez de compilar novamente.

Aí vem nosso primeiro truque para acelerar o tempo de carregamento do tempo de execução

Na primeira vez, iremos:

Clone e compile o projeto darknet.
Copie a versão compilada para o nosso Google Drive Floder
Nas próximas vezes, em vez de compilar novamente a cada carregamento de tempo de execução, copiaremos a versão compilada para nossa máquina VM!

Ao compilar pela primeira vez, sua última linha de saída deve ser algo como isto:

g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCVpkg-config --cflags opencv-DGPU (...)

PASSO 4-A. Clonando e compilando Darkent. SÓ PRECISA SER EXECUTO NA PRIMEIRA EXECUÇÃO!!
Nesta etapa, vamos clonar o repositório darkent e compilá-lo.

Clonar repositório
Compilar Darknet
Copie a versão compilada para o Drive
Quando a compilação terminar, sua última linha de saída deve ser algo como isto:

_ g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCVpkg-config --cflags opencv-DGPU (...)_

[ ]
# Leave this code uncommented on the very first run of your notebook or if you ever need to recompile darknet again.
# Comment this code on the future runs.
!git clone https://github.com/kriyeng/darknet/
%cd darknet

# Check the folder
!ls

# I have a branch where I have done the changes commented above
!git checkout feature/google-colab

#Compile Darknet
!make

#Copies the Darknet compiled version to Google drive
!cp ./darknet /content/gdrive/My\ Drive/darknet/bin/darknet
PASSO 4-B. Copiando a versão compilada do Darknet do Drive. DESCOMENTAR APÓS A PRIMEIRA EXECUÇÃO
Copie a versão compilada da darknet da unidade para a VM.

Faça a pasta darknet local
Copie o arquivo darknet
Definir permissões de execução
[ ]
# Uncomment after the first run, when you have a copy of compiled darkent in your Google Drive

# Makes a dir for darknet and move there
#!mkdir darknet
#%cd darknet

# Copy the Darkent compiled version to the VM local drive
#!cp /content/gdrive/My\ Drive/darknet/bin/darknet ./darknet

# Set execution permissions to Darknet
#!chmod +x ./darknet

ETAPA 5. Configuração do tempo de execução concluída!
Vamos conferir!

Se você estiver executando este notebook pela primeira vez, poderá executar as células a seguir para verificar se tudo ocorre conforme o esperado!

Alguns Utilitários
Vamos adicionar alguns utilitários que talvez possam ser úteis.

Esses utilitários são:

imgShow() - Nos ajudará a mostrar uma imagem na VM remota
download() - Vai permitir que você pegue algum arquivo do seu notebook caso precise
upload () - Você pode fazer upload de arquivos para sua pasta atual na VM remota.
Obrigado a Ivan Goncharov por esses ajudantes!

[ ]
#download files
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)
[ ]
# Not necessary cell
# Get yolov3 weights
!wget https://pjreddie.com/media/files/yolov3.weights
NOTA: O teste a seguir só funcionará quando a darknet for compilada no tempo de execução. Esta demonstração usa alguns dados das pastas originais da darknet. Para seus projetos de detecção de objetos, você terá esses arquivos necessários em sua pasta local.

[ ]
# Not necessary cell
# Execute darknet using YOLOv3 model with pre-trained weights to detect objects on 'person.jpg'
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg -dont-show

# Show the result using the helper imgShow()
imShow('predictions.jpg')
Se você pode ver a mesma foto abaixo, parabéns!! Neste ponto você tem o Darknet configurado e funcionando!
pessoa.jpg

PARTE 2. Treinamento YOLO
TRUQUE: Toda vez que você quiser executar todas as suas células automaticamente, você pode ir para > Menu > Runtime > executar tudo . Talvez você não queira executar todo o notebook. Você pode escrever a seguinte célula onde deseja parar o processo e descomentar o assert Falsecomando. Isso gerará um erro e interromperá a execução de mais células. Agradecimentos a: Este tópico

[ ]
# don't go beyond here with Run All
#assert False
PARTE 2. PASSO 0. Preparando seus dados e arquivos de configuração
Antes de prosseguir, vamos dar uma olhada em quais arquivos de configuração você precisa ter em sua unidade localdarknet

Ficheiros de configuração do Yolov3 cheat sheet.jpg

Você pode baixar a folha de dicas aqui

Se você precisar de explicações mais profundas sobre como preparar seus conjuntos de dados, anotação e aprendizado profundo, visite [How to train YOLOv3 using Darknet framework and optimize the VM runtime load times](post link)

TRUQUE: Você deve configurar cuidadosamente os caminhos em seus arquivos de configuração. O arquivo obj.data precisa ter espaços no caminho com \ . Assim: /content/gdrive/My \ Drive/darknet/train.txt . Mas, nos arquivos train.txt e test.txt não!

PARTE 2. ETAPA 1. Carregando arquivos na unidade local da VM
A velocidade da rede entre o Google Drive e a VM do Colab pode ter um impacto na velocidade de treinamento ao acessar as imagens do conjunto de dados.

Você pode ter 3 abordagens possíveis, dependendo do tamanho do seu conjunto de dados.

NOTA: Esta etapa não é necessária para todos os arquivos de configuração e pesos. Eles podem ser acessados ​​diretamente do Google Drive sem problemas consideráveis ​​de desempenho. Esta etapa é apenas para as imagens e anotações do conjunto de dados

Opção 1 - Você pode tentar usar diretamente os arquivos da img/pasta do Google Drive. Dependendo do seu conjunto de dados, talvez isso possa ser mais do que bom.
Opção 2 - Antes de iniciar o treinamento, copie seu conjunto de dados do Google Drive para o sistema de arquivos da VM local. Talvez seja uma boa prática copiar como um único arquivo tar e descompactar na img/pasta local da VM
Opção 3 - Se o seu conjunto de dados for muito grande, talvez você possa fazer upload para um repositório git e clonar daqui. Normalmente, o tempo de transferência entre eles é muito melhor. Se você tiver que decidir, tenho a sensação de que o bitbucket tem tempos de transferência de velocidade melhores do que o github, mas, por favor, não tome isso como confirmado, não fiz testes específicos sobre isso, posso estar errado!
PARTE 2. PASSO 1 - Opção 1. Usando arquivos do Google Drive diretamente.
Você não precisa fazer nada aqui. Seu train.txt deve ter o caminho correto:

/content/grdive/My Drive/darknet/img/image001.jpg . Como dito antes, não use espaços em branco com escape para os caminhos em train.txt e test.txt
PARTE 2. PASSO 1 - Opção 2A. Copiando arquivos do Google Drive para o sistema de arquivos local da VM.
Execute a seguinte célula para copiar seus arquivos

[ ]
# Copy fils from Google Drive to the VM local filesystem
!cp -r "/content/gdrive/My Drive/darknet/img" ./img

PARTE 2. PASSO 1 - Opção 2B. Copiar arquivos compactados do Google Drive para o sistema de arquivos local da VM e descompactá-los localmente.
Execute a célula a seguir para copiar seus arquivos e descompactá-los. Você pode usar o comando !ls para descobrir qual é o caminho correto que você deve configurar em seu train.txt para acessar corretamente as imagens do conjunto de dados

[ ]
# Copy your compressed file
#!cp -r "/content/gdrive/My Drive/darknet/img/img.tgz" ./img

# Uncompress zipped file
#!tar -xzvf ./img/img.tgz
PARTE 2. PASSO 1 - Opção 3. Clone seu conjunto de dados de imagem de um repositório git. Parece o mais rápido.
Execute a seguinte célula para clonar seu repositório de conjunto de dados para o sistema de arquivos local da VM

[ ]
# Git clone directly to ./img folder
#!git clone https://[your-repository] ./img

# Check the result - Uncomment when you checked for speed up further runs
#!ls -la ./img
PARTE 3. Finalmente, treine seu modelo!
Ao executar o seguinte comando, seu modelo começará a treinar.

Você terá uma linha de log por época. Em cada iteração, você verá como está indo seu treinamento.

TRUQUE: Darknet copia um backup de seus pesos treinados a cada 100 iterações. Como num passe de mágica, esse arquivo será sincronizado com o drive local do seu computador, assim como os backups darknet fazem a cada 1000 iterações, salvando-o em um arquivo separado.

[ ]
!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3.cfg" "/content/gdrive/My Drive/darknet/darknet53.conv.74" -dont_show 
DICAS E TRUQUES DE DESEMPENHO
Acelere os tempos de carregamento do tempo de execução: quando tudo estiver funcionando corretamente, você pode remover células ou comentar linhas de código desnecessárias para diminuir o tempo de carregamento a cada execução.

Como manter seu notebook ativo por mais tempo?: Mantenha seu navegador com o notebook aberto. Se você fechar o navegador, seu notebook atingirá o tempo ocioso e será removido do Colab Cloud Service. (90 minutos)

Execute novamente seu treinamento após atingir o tempo de limitação para os tempos de execução do Colab (12 horas):

Abra um novo notebook ou reconecte o atual.
Comente a célula acima e descomente a célula abaixo.
Em seu computador local, copie o arquivo backup/yolov3_last.weights para a pasta pesos/ do computador local .
Execute Executar tudo no > menu > Tempo de execução > Executar tudo
A etapa de cópia não é absolutamente necessária, mas gosto de manter uma cópia da última sessão de treinamento e não sobrescrever este arquivo nos próximos treinamentos.
[ ]
# Start training at the point where the last runtime finished
#!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3.cfg" "/content/gdrive/My Drive/darknet/weights/yolov_last.weights" -dont_show 
SOLUÇÃO DE PROBLEMAS
Os principais problemas que você pode enfrentar se o seu modelo gerar um erro são:

Imagens ou arquivos não encontrados. Verifique a imagem da folha de dicas do Yolov3 acima e verifique se está tudo bem.
Se você configurou incorretamente seus filtros e classes no yolov3.cfg . Verifique a folha de checagem Yolov3 acima.
Você pode enfrentar alguns erros de falta de memória ou biblioteca principalmente pela falta de alguns dos pré-requisitos. Nesse caso, verifique as versões das bibliotecas atuais instaladas em sua VM do Colab. Você pode encontrar mais informações nas primeiras etapas deste notebook.
Parâmetros de lote e subdivisões em seu yolov3.cfg também podem afetar a alocação de memória. Consulte o repositório original Aqui para obter mais detalhes.
ATUALIZAÇÃO DE SOLUÇÃO DE PROBLEMAS
Tenha cuidado se estiver preparando seus arquivos no Windows. Se você usar CRLF em seus arquivos em vez de LF , poderá ter problemas para abrir os arquivos corretamente. - Obrigado a Satya Shetty por compartilhar este problema!

Sobre mim
Você pode encontrar a postagem original com mais explicações sobre este notebook em [How to train YOLOv3 using Darknet framework and optimize the VM runtime load times](post link)

Sou David Ibañez, de Barcelona. Sinta-se a vontade para entrar em contato!

Você pode visitar meu blog em Dev-ibanyez.info
Você pode entrar em contato comigo no Twitter
Você pode entrar em contato ou contribuir com este notebook no Github
Você pode comentar no post dev.to sobre este notebook
Obrigado por ter lido este caderno! :palmas: :palmas: :palmas:

FONTES
Outras fontes
Site original da YOLO Joseph Redmon Page
AlexeyAB repositório darknet github
O notebook Ivan Goncharov me inspirou a experimentar o Google Colab e acabar criando este notebook.
